In [1]:
import numpy as np

### 1) Thermodynamic Analysis of an Ideal Mixture Containing (CH$_4$, CO$_2$, H$_2$O, H$_2$, CO, Ar)
### 2) Thermodynamic Analysis of 4 Parallel DRM Reactions (i.e., DRM, SRM1, SRM2, WGS) 

### Input Data

In [2]:
# coefficients of shomate equation - nist data from https://webbook.nist.gov/

co2 = [24.99735, 55.18696, -33.69137, 7.948387, -0.136638, -403.6075, 228.2431, -393.5224]
ch4 = [-0.703029, 108.4773, -42.52157, 5.862788, 0.678565, -76.84376, 158.7163, -74.87310]
h2o = [30.09200, 6.832514, 6.793435, -2.534480, 0.082139, -250.8810, 223.3967, -241.8264]
h2 = [33.066178, -11.363417, 11.432816, -2.772874, -0.158558, -9.980797, 172.707974, 0.0]
co = [25.56759, 6.096130, 4.054656,  -2.671301,  0.131021, -118.0089, 227.3665, -110.5271]
ar = [20.78600, 2.825911e-07, -1.464191e-07,  1.092131e-08, -3.661371e-08, -6.197350, 179.9990, 0.00000]

######################################################################################################################
# enthalpy of formation in kJ/mol
HF298_ch4 = -74.85
HF298_co2 = -393.51
HF298_h2o = -241.826
HF298_h2 = 0.0
HF298_co = -110.53
HF298_ar = 0.0

######################################################################################################################
# entropy of formation in J/mol/K
SF298_ch4 = 186.25
SF298_co2 = 197.66
SF298_h2o = 188.84
SF298_h2 = 130.68
SF298_co = 213.79
SF298_ar = 154.84

# gibbs of formation in kJ/mol
GF298_ch4 = HF298_ch4 - 298.15*SF298_ch4/1000
GF298_co2 = HF298_co2 - 298.15*SF298_co2/1000
GF298_h2o = HF298_h2o - 298.15*SF298_h2o/1000
GF298_h2 = HF298_h2 - 298.15*SF298_h2/1000
GF298_co = HF298_co - 298.15*SF298_co/1000
GF298_ar = HF298_ar - 298.15*SF298_ar/1000

######################################################################################################################
# molar masses of the species containing in the mixture in kg/mol
mr_ch4 = 16.04e-03
mr_co2 = 44.01e-03
mr_h2o = 18.01528e-03
mr_h2 = 2.01588e-03
mr_co = 28.01e-03
mr_ar = 39.948e-03

### Equations

In [3]:
# assumption of an ideal gas mixture

# partial concentrations of species in mol/m3
def ci_mol_m3(T, P, y):

    return np.array([y*P/(8.314e-05*T)])

# total concentration of the mixture in mol/m3
def c_total_mol_m3(T, P, y):

    return np.array([np.sum(ci_mol_m3(T, P, y))])

# density of the mixture in kg/m3
def d_mix_kg_m3(T, P, y):

    return np.array([c_total_mol_m3(T, P, y)*Mr.dot(y)])

# molar mass of the mixture in kg/mol
def mr_mix_kg_mol(Mr, y):

    return np.array([Mr.dot(y)])

######################################################################################################################

# specific heat capacity of species in J/mol/K
def cpi_j_mol_K(T, WBi):

    t = T/1000

    T_Cpi = np.array([1,  t, t**2, t**3, 1.0/t])
    
    return np.array([WBi.dot(T_Cpi)])

# specific heat capacity of the mixture in J/mol/K
def cp_mix_j_mol_K(T, y, WBi):

    t = T/1000

    T_Cpi = np.array([1,  t, t**2, t**3, 1.0/t])
    
    return np.array([WBi.dot(T_Cpi).dot(y)])

# specific heat capacity of the mixture in J/kg/K
def cp_mix_kg_mol_K(T, y, WBi, Mr):

    t = T/1000

    T_Cpi = np.array([1,  t, t**2, t**3, 1.0/t])
    
    num = (WBi.dot(T_Cpi)).dot(y)
    den = Mr.dot(y)
    
    return np.array([num/den])

######################################################################################################################
# enthalpy of reaction in kJ/mol
def enthalpy_rxn_kj_mol(T, WBi, HF298_j, stoichiometry):

    t = T/1000

    Hrxn298 = np.dot(HF298_j, stoichiometry)

    T_Hi = np.array([t,  t**2 / 2.0, t**3 / 3.0, t**4 / 4.0, -1.0 / t, 1.0, 0.0, -1.0])

    Hrxn = Hrxn298 + (WBi.dot(T_Hi)).dot(stoichiometry)

    return np.array([Hrxn])

# partial enthalpies of species in kJ/mol
def partial_enthalpies_kj_mol(T, WBi, HF298_i):

    t = T/1000

    T_Hi = np.array([t,  t**2 / 2.0, t**3 / 3.0, t**4 / 4.0, -1.0 / t, 1.0, 0.0, -1.0])

    Hi = WBi.dot(T_Hi) # (H - H_298.15) kJ/mol

    return np.array([(Hi + HF298_i)])

######################################################################################################################
# entropy of reaction in kJ/mol
def entropy_rxn_kj_mol(T, WBi, SF298_j, stoichiometry):

    t = T/1000

    Srxn298 = np.dot(SF298_j, stoichiometry)

    T_Si = np.array([np.log(t), t,  t**2 / 2.0,  t**3 / 3.0, -1.0 / (2.0 * t**2), 0.0, 1.0, 0.0])
    
    Srxn = (WBi.dot(T_Si/1000.0)).dot(stoichiometry)

    return np.array([Srxn])

######################################################################################################################
def gibbs_rxn_kj_mol(T, WBi, HF298_j, SF298_j, stoichiometry):

    Grxn = enthalpy_rxn_kj_mol(T, WBi, HF298_j, stoichiometry) - T*entropy_rxn_kj_mol(T, WBi, SF298_j, stoichiometry)

    return np.array([Grxn])

In [4]:
# condition for thermodynamic analysis for both cases
T = 873.15 # K
P = 1 # bar
R = 8.314e-3 # kJ/mol/K

### 1) Mixture

In [5]:
######################################################################################################################
# mixture of CH4:0.3, CO2:0.2, H2O:0.0, H2:0.0, CO:0.0, Ar:0.5
WBi_species = np.array([ch4, co2, h2o, h2, co, ar])

# heats of formation at 298.15 K for CH4, CO2, H2O, H2, CO, Ar in kJ/mol
HF298_species = np.array([HF298_ch4, HF298_co2, HF298_h2o,  HF298_h2, HF298_co, HF298_ar])

# molar mass of species in the mixture in kg/mol
Mr = np.array([mr_ch4, mr_co2, mr_h2o, mr_h2, mr_co, mr_ar])

# molar ratio of the species in the mixture dimensionless
y = np.array([0.3, 0.2, 0.0, 0.0, 0.0, 0.5])

In [6]:
# partial molar enthalpies of the mixture in mol/m3
ci_mol_m3(T, P, y)

array([[4.13259053, 2.75506035, 0.        , 0.        , 0.        ,
        6.88765088]])

In [7]:
# total concentration of the mixture in mol/m3
c_total_mol_m3(T, P, y)

array([13.77530177])

In [8]:
# density of the mixture in kg/m3
d_mix_kg_m3(T, P, y)

array([[0.46268484]])

In [9]:
# molar mass of the mixture in kg/mol
mr_mix_kg_mol(Mr, y)

array([0.033588])

In [10]:
# specific heat capacity of species in J/mol/K
cpi_j_mol_K(T, WBi_species[:,:-3])

array([[66.27576278, 52.63245614, 39.64397803, 29.83304121, 32.35347837,
        20.7860001 ]])

In [11]:
# specific heat capacity of the mixture in J/mol/K
cp_mix_j_mol_K(T, y, WBi_species[:,:-3])

array([40.80222011])

In [12]:
# specific heat capacity of the mixture in J/kg/K
cp_mix_kg_mol_K(T, y,  WBi_species[:,:-3], Mr)

array([1214.78564105])

In [13]:
# partial enthalpies of species in the gas mixture in kJ/mol
partial_enthalpies_kj_mol(T, WBi_species, HF298_species)

array([[ -45.44395579, -366.89608539, -220.95618251,   16.87479203,
         -93.0021606 ,   11.95194602]])

### 2) Parallel Reactions

In [14]:
# CO2 + CH4 ⇌ 2CO + 2H2 - DRM reaction
WBi_DRM = np.array([co2, ch4, co, h2])

# heats of formation at 298.15 K for CO2, CH4, CO, H2
HF298_DRM = np.array([HF298_co2, HF298_ch4, HF298_co, HF298_h2]) # kJ/mol
SF298_DRM = np.array([SF298_co2, SF298_ch4, SF298_co, SF298_h2]) # J/mol/K

stoichiometry_DRM = np.array([-1, -1, 2, 2])

######################################################################################################################
# CH4 + H2O ⇌ CO + 3H2 - SRM1 reaction
WBi_SRM1 = np.array([ch4, h2o, co, h2])

# heats of formation at 298.15 for CH4, H2O, CO, H2
HF298_SRM1 = np.array([HF298_ch4, HF298_h2o, HF298_co, HF298_h2]) # kJ/mol
SF298_SRM1 = np.array([SF298_ch4, SF298_h2o, SF298_co, SF298_h2]) # J/mol/K

stoichiometry_SRM1 = np.array([-1, -1, 1, 3])

######################################################################################################################
# CH4 + 2H2O ⇌ CO2 + 4H2 - SRM2 reaction
WBi_SRM2 = np.array([ch4, h2o, co2, h2])

# heats of formation at 298.15 K for CH4, H2O, CO2, H2
HF298_SRM2 = np.array([HF298_ch4, HF298_h2o, HF298_co2, HF298_h2]) # kJ/mol
SF298_SRM2 = np.array([SF298_ch4, SF298_h2o, SF298_co2, SF298_h2]) # J/mol/K

stoichiometry_SRM2 = np.array([-1, -2, 1, 4])

######################################################################################################################
# CO + H2O ⇌ CO2 + H2 - WGS reaction
WBi_WGS = np.array([co, h2o, co2, h2])

# heats of formation at 298.15 K for CO, H2O, CO2, H2
HF298_WGS = np.array([HF298_co, HF298_h2o, HF298_co2, HF298_h2]) # kJ/mol
SF298_WGS = np.array([SF298_co, SF298_h2o, SF298_co2, SF298_h2]) # J/mol/K

stoichiometry_WGS = np.array([-1, -1,  1,  1])

In [15]:
# enthalpy of DRM reaction in kJ/mol
enthalpy_rxn_kj_mol(T, WBi_DRM, HF298_DRM, stoichiometry_DRM)

array([260.08530404])

In [16]:
# enthalpy of SRM1 reaction in kJ/mol
enthalpy_rxn_kj_mol(T, WBi_SRM1, HF298_SRM1, stoichiometry_SRM1)

array([224.0223538])

In [17]:
# enthalpy of SRM2 reaction in kJ/mol
enthalpy_rxn_kj_mol(T, WBi_SRM2, HF298_SRM2, stoichiometry_SRM2)

array([187.95940355])

In [18]:
# enthalpy of WGS reaction in kJ/mol
enthalpy_rxn_kj_mol(T, WBi_WGS, HF298_WGS, stoichiometry_WGS)

array([-36.06295024])

In [19]:
# entropy of DRM reaction in kJ/mol
entropy_rxn_kj_mol(T, WBi_DRM, SF298_DRM, stoichiometry_DRM)

array([0.28424661])

In [20]:
# entropy of SRM1 reaction in kJ/mol
entropy_rxn_kj_mol(T, WBi_SRM1, SF298_SRM1, stoichiometry_SRM1)

array([0.25110111])

In [21]:
# entropy of SRM2 reaction in kJ/mol
entropy_rxn_kj_mol(T, WBi_SRM2, SF298_SRM2, stoichiometry_SRM2)

array([0.21795562])

In [22]:
# entropy of SRM2 reaction in kJ/mol
entropy_rxn_kj_mol(T, WBi_WGS, SF298_WGS, stoichiometry_WGS)

array([-0.0331455])

In [23]:
# gibbs energy of DRM reaction in kJ/mol
gibbs_rxn_kj_mol(T, WBi_DRM, HF298_DRM, SF298_DRM, stoichiometry_DRM)

array([[11.89537936]])

In [24]:
# gibbs energy of SRM1 reaction in kJ/mol
gibbs_rxn_kj_mol(T, WBi_SRM1, HF298_SRM1, SF298_SRM1, stoichiometry_SRM1)

array([[4.77341847]])

In [25]:
# gibbs energy of SRM2 reaction in kJ/mol
gibbs_rxn_kj_mol(T, WBi_SRM2, HF298_SRM2, SF298_SRM2, stoichiometry_SRM2)

array([[-2.34854241]])

In [26]:
# gibbs energy of WGS reaction in kJ/mol
gibbs_rxn_kj_mol(T, WBi_WGS, HF298_WGS, SF298_WGS, stoichiometry_WGS)

array([[-7.12196089]])

In [27]:
# Keq of DRM reaction
np.exp(-gibbs_rxn_kj_mol(T, WBi_DRM, HF298_DRM, SF298_DRM, stoichiometry_DRM)/R/T)

array([[0.19424706]])

In [28]:
# Keq of SRM1 reaction
np.exp(-gibbs_rxn_kj_mol(T, WBi_SRM1, HF298_SRM1, SF298_SRM1, stoichiometry_SRM1)/R/T)

array([[0.51811772]])

In [29]:
# Keq of SRM2 reaction
np.exp(-gibbs_rxn_kj_mol(T, WBi_SRM2, HF298_SRM2, SF298_SRM2, stoichiometry_SRM2)/R/T)

array([[1.38198214]])

In [30]:
# Keq of WGS reaction
np.exp(-gibbs_rxn_kj_mol(T, WBi_WGS, HF298_WGS, SF298_WGS, stoichiometry_WGS)/R/T)

array([[2.66731301]])